In [49]:
import pandas as pd
import os
import duckdb

In [50]:
from dotenv import load_dotenv, find_dotenv
load_dotenv('/workspaces/patient_conditions_weather/src/.env')

True

In [51]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/workspaces/patient_conditions_weather/airflow/credentials/google_credentials.json'
emr_path = f"gs://{os.getenv('EMR_RAW_BUCKET')}/{os.getenv('EMR_RAW_PATH')}"
encounters_path = emr_path +'/encounters.csv'
organizations_path = emr_path + '/organizations.csv'
conditions_path = emr_path + '/conditions.csv'


In [52]:
encounters = pd.read_csv(encounters_path, delimiter=",")
conditions = pd.read_csv(conditions_path, delimiter=",")
organizations = pd.read_csv(organizations_path, delimiter=",")





In [53]:

conditions

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2013-06-24,2013-07-02,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,0b2794bd-ec2b-d34f-0610-2523b3b7fcf0,10509002,Acute bronchitis (disorder)
1,2016-02-27,2016-03-14,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,a6d818dd-0983-fd1c-eefa-3d2295532c45,283371005,Laceration of forearm
2,2016-08-11,2016-08-22,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,36d2e781-4655-0a11-1f70-c69856e02019,444814009,Viral sinusitis (disorder)
3,2016-11-27,2016-12-17,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,c8eaaf41-958b-31ab-7de5-568cee8751f3,444814009,Viral sinusitis (disorder)
4,2017-02-22,2017-06-02,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,6474f606-5a1b-48c0-bbbf-ad6dcbc24d4e,16114001,Fracture of ankle
...,...,...,...,...,...,...
38089,2020-01-10,2021-01-15,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,090096d3-9404-3cb5-d1eb-583bf4c39180,160903007,Full-time employment (finding)
38090,2020-12-27,2021-01-03,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,db101ad8-66e2-9feb-e0cf-b2618f873c3a,38822007,Cystitis
38091,2021-01-15,NaN,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,1516d2e6-4846-5f1e-fe27-c1ebb9a39f72,160904001,Part-time employment (finding)
38092,2021-01-15,NaN,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,1516d2e6-4846-5f1e-fe27-c1ebb9a39f72,73595000,Stress (finding)


In [74]:
daily_data_path = f"gs://{os.getenv('WEATHER_BUCKET')}/{os.getenv('WEATHER_DATA_DAILY_PATH')}"



daily_data = pd.read_parquet(daily_data_path, engine="pyarrow")
daily_data

,date,weather_code,temperature_2m_mean,apparent_temperature_mean,daylight_duration,sunshine_duration,frequency,year,latitude,longitude
0,1979-01-01 00:00:00+00:00,51,7.9,3.4,33198.72,1251.11,daily,1979,41.6,-70.5
1,1979-01-02 00:00:00+00:00,61,11.9,7.1,33243.27,1247.72,daily,1979,41.6,-70.5
2,1979-01-03 00:00:00+00:00,63,4.6,-1.0,33291.53,28647.03,daily,1979,41.6,-70.5
3,1979-01-04 00:00:00+00:00,3,-4.4,-10.5,33343.43,24297.17,daily,1979,41.6,-70.5
4,1979-01-05 00:00:00+00:00,3,-3.3,-8.9,33398.93,29351.38,daily,1979,41.6,-70.5
...,...,...,...,...,...,...,...,...,...,...
104492,1982-12-27 00:00:00+00:00,3,3.6,-0.5,33034.32,25317.30,daily,1982,41.6,-71
104493,1982-12-28 00:00:00+00:00,3,8.7,5.5,33059.29,10408.31,daily,1982,41.6,-71
104494,1982-12-29 00:00:00+00:00,55,11.3,8.2,33088.24,14400.00,daily,1982,41.6,-71
104495,1982-12-30 00:00:00+00:00,51,4.0,0.1,33121.12,26484.97,daily,1982,41.6,-71


In [28]:
import re
rexp = r'\((.*?)\)'
#conditions['category'] = conditions['DESCRIPTION'].apply(lambda x: re.search(rexp, x).group(1))
conditions['condition_name'] = conditions['DESCRIPTION'].apply(lambda x: re.sub(r' \([^)]*\)', '', x))
conditions['condition_type'] = conditions['DESCRIPTION'].apply(lambda x: re.search(rexp, x).group(1) if re.search(rexp, x) else '')
conditions

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,condition_name,condition_type
0,2013-06-24,2013-07-02,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,0b2794bd-ec2b-d34f-0610-2523b3b7fcf0,10509002,Acute bronchitis (disorder),Acute bronchitis,disorder
1,2016-02-27,2016-03-14,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,a6d818dd-0983-fd1c-eefa-3d2295532c45,283371005,Laceration of forearm,Laceration of forearm,
2,2016-08-11,2016-08-22,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,36d2e781-4655-0a11-1f70-c69856e02019,444814009,Viral sinusitis (disorder),Viral sinusitis,disorder
3,2016-11-27,2016-12-17,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,c8eaaf41-958b-31ab-7de5-568cee8751f3,444814009,Viral sinusitis (disorder),Viral sinusitis,disorder
4,2017-02-22,2017-06-02,c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8,6474f606-5a1b-48c0-bbbf-ad6dcbc24d4e,16114001,Fracture of ankle,Fracture of ankle,
...,...,...,...,...,...,...,...,...
38089,2020-01-10,2021-01-15,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,090096d3-9404-3cb5-d1eb-583bf4c39180,160903007,Full-time employment (finding),Full-time employment,finding
38090,2020-12-27,2021-01-03,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,db101ad8-66e2-9feb-e0cf-b2618f873c3a,38822007,Cystitis,Cystitis,
38091,2021-01-15,NaN,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,1516d2e6-4846-5f1e-fe27-c1ebb9a39f72,160904001,Part-time employment (finding),Part-time employment,finding
38092,2021-01-15,NaN,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,1516d2e6-4846-5f1e-fe27-c1ebb9a39f72,73595000,Stress (finding),Stress,finding


In [29]:
# Create a DuckDB connection
con = duckdb.connect()
con.register('encounters', encounters)
con.register('conditions', conditions)
con.register('organizations', organizations)

In [55]:
query = """
with events as (
    select 
        CAST(e.stop AS DATE) diagnosis_date,
        o.name, o.city,
        c.code,
        c.condition_name,
        c.condition_type,
        round(o.LAT,1) as latitude,
        round(o.LON,1) as longitude
    from encounters e
    inner join organizations o on e.ORGANIZATION = o.id
    inner join conditions c on e.id = c.ENCOUNTER
)
select *
from events
where extract('year' from diagnosis_date) > 1980
and condition_type in ('disorder', '')
-- and latitude = 41.6	
-- and longitude = -71.1
order by 1
"""
diagnoses = con.execute(query).fetchdf()
#full count conditions join encounters = 38094
diagnoses

,diagnosis_date,NAME,CITY,CODE,condition_name,condition_type,latitude,longitude
0,1981-01-09,PCP19822,MARBLEHEAD,44054006,Diabetes,,42.5,-70.8
1,1981-01-09,NORTH SHORE MEDICAL CENTER -,SALEM,59621000,Hypertension,,42.5,-70.9
2,1981-01-09,PCP19822,MARBLEHEAD,271737000,Anemia,disorder,42.5,-70.8
3,1981-01-15,APPLETON EYE ASSOCIATES PC,AMESBURY,44054006,Diabetes,,42.9,-71.0
4,1981-01-15,APPLETON EYE ASSOCIATES PC,AMESBURY,271737000,Anemia,disorder,42.9,-71.0
...,...,...,...,...,...,...,...,...
7214,2021-11-13,HOLYOKE MEDICAL CENTER,HOLYOKE,72892002,Normal pregnancy,,42.2,-72.6
7215,2021-11-13,Edith Nourse Rogers Memorial Veterans Hospital...,Bedford,43878008,Streptococcal sore throat,disorder,42.5,-71.3
7216,2021-11-14,BRIGHAM AND WOMEN'S FAULKNER HOSPITAL,BOSTON,62106007,Concussion with no loss of consciousness,,42.3,-71.0
7217,2021-11-15,CAREWELL URGENT CARE CENTERS OF MA PC,LEXINGTON,59621000,Hypertension,,42.4,-71.2


In [ ]:
con.register('diagnoses', diagnoses)

In [78]:
query = """
with weather as (
    select 
        cast(date as date) as weather_date,
        weather_code,
        temperature_2m_mean,	apparent_temperature_mean,	daylight_duration,	sunshine_duration,	latitude,	longitude
    from daily_data     
)
select 
    d.diagnosis_date,
    d.code,
    d.condition_name,
    d.condition_type,
    w.daylight_duration,
    w.sunshine_duration,
    w.apparent_temperature_mean    
from diagnoses d 
inner join weather w on 
    w.latitude = d.latitude
    and w.longitude = d.longitude 
    and w.weather_date = d.diagnosis_date
"""
diagnoses_with_weather = con.execute(query).fetchdf()

diagnoses_with_weather

,diagnosis_date,CODE,condition_name,condition_type,daylight_duration,sunshine_duration,apparent_temperature_mean
0,1982-06-27,10509002,Acute bronchitis,disorder,54512.88,45964.85,14.3
1,1982-05-15,185086009,Chronic obstructive bronchitis,disorder,52255.92,25003.58,9.4
2,1981-02-20,43878008,Streptococcal sore throat,disorder,39058.51,3875.94,5.5
3,1981-04-03,15777000,Prediabetes,,46066.02,41434.92,5.2
4,1981-04-02,33737001,Fracture of rib,,45910.48,31331.97,4.7
5,1981-03-07,15777000,Prediabetes,,41487.61,0.00,-4.4
6,1981-08-08,195662009,Acute viral pharyngitis,disorder,50993.21,26263.54,22.1
7,1981-10-21,271737000,Anemia,disorder,39028.48,35614.90,8.4
8,1981-12-17,444814009,Viral sinusitis,disorder,32877.75,25728.16,-6.3
9,1981-03-02,59621000,Hypertension,,40628.02,30375.84,-1.2


In [69]:
daily_data[daily_data['date'] == '1981-07-31 00:00:00+00:00']

,date,weather_code,temperature_2m_mean,apparent_temperature_mean,daylight_duration,sunshine_duration,frequency,year,latitude,longitude


In [75]:
daily_data[daily_data['date'] == '1981-07-01 00:00:00+00:00']

,date,weather_code,temperature_2m_mean,apparent_temperature_mean,daylight_duration,sunshine_duration,frequency,year,latitude,longitude
69273,1981-07-01 00:00:00+00:00,3,16.8,16.5,54388.58,36931.49,daily,1981,41.3,-70.1
69638,1981-07-01 00:00:00+00:00,51,18.2,19.3,54502.95,36001.70,daily,1981,41.6,-70.5
70003,1981-07-01 00:00:00+00:00,51,19.6,21.0,54502.95,36001.70,daily,1981,41.6,-70.6
70368,1981-07-01 00:00:00+00:00,51,20.1,21.8,54502.95,36168.18,daily,1981,41.6,-70.9
70733,1981-07-01 00:00:00+00:00,51,19.1,20.4,54502.95,35676.78,daily,1981,41.6,-71.1
...,...,...,...,...,...,...,...,...,...,...
101028,1981-07-01 00:00:00+00:00,3,19.3,20.9,54676.82,34185.70,daily,1981,42,-70.9
101393,1981-07-01 00:00:00+00:00,3,19.9,21.6,54676.82,34240.60,daily,1981,42,-71.1
101758,1981-07-01 00:00:00+00:00,3,20.4,22.2,54676.82,34710.33,daily,1981,42,-71.2
102123,1981-07-01 00:00:00+00:00,3,20.9,22.6,54676.82,35188.21,daily,1981,42,-71.5
